In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

# Initialisation du DataFrame global
df = pd.DataFrame(columns=['Vb (mL)', 'pH'])

# Widgets pour entrée manuelle
vb_input      = widgets.FloatText(description='Vb (mL):', value=0.0)
ph_input      = widgets.FloatText(description='pH:', value=7.0)
add_button    = widgets.Button(description='Ajouter', button_style='success')
clear_button  = widgets.Button(description='Réinitialiser', button_style='warning')

# Widgets pour import Excel
upload        = widgets.FileUpload(accept='.xlsx, .xls', multiple=False)
import_button = widgets.Button(description='Importer Excel', button_style='info')

# Checkbox pour la dérivée
show_deriv    = widgets.Checkbox(description="Afficher dérivée d(pH)/dV", value=False)

# Zones d'affichage
output_table  = widgets.Output()
output_plot   = widgets.Output()

def update_outputs():
    with output_table:
        clear_output(wait=True)
        display(df)
    with output_plot:
        clear_output(wait=True)
        if df.empty:
            print("Pas de données à tracer.")
            return
        vb = df['Vb (mL)'].astype(float).values
        ph = df['pH'].astype(float).values
        ordre = np.argsort(vb)
        vb, ph = vb[ordre], ph[ordre]
        dpH_dV = np.gradient(ph, vb)
        idx_inf = np.argmax(np.abs(dpH_dV))
        V_eq, pH_eq = vb[idx_inf], ph[idx_inf]

        fig, ax1 = plt.subplots(figsize=(6, 4))
        ax1.plot(vb, ph, 'o-', label="pH")
        ax1.axvline(V_eq, linestyle='--', label=f"Vₑq ≈ {V_eq:.2f} mL")
        ax1.plot([V_eq], [pH_eq], 'ro')
        ax1.set_xlabel("Vb (mL)")
        ax1.set_ylabel("pH")
        ax1.set_ylim(bottom=0)
        ax1.grid(True)

        if show_deriv.value:
            ax2 = ax1.twinx()
            ax2.plot(vb, dpH_dV, 'x--', label="d(pH)/dV")
            ax2.set_ylabel("d(pH)/dV")
            h1, l1 = ax1.get_legend_handles_labels()
            h2, l2 = ax2.get_legend_handles_labels()
            ax1.legend(h1 + h2, l1 + l2, loc='best')
        else:
            ax1.legend(loc='best')

        plt.show()
        print(f"Point d'équivalence : Vₑq ≈ {V_eq:.2f} mL, pH ≈ {pH_eq:.2f}")

def on_add_clicked(b):
    global df
    try:
        vb = float(vb_input.value)
        ph = float(ph_input.value)
    except ValueError:
        return
    df.loc[len(df)] = {'Vb (mL)': vb, 'pH': ph}
    vb_input.value = 0.0
    ph_input.value = 7.0
    update_outputs()

def on_clear_clicked(b):
    global df
    df = pd.DataFrame(columns=['Vb (mL)', 'pH'])
    update_outputs()

def on_import_clicked(b):
    global df
    if upload.value:
        # upload.value peut être un tuple (widgets v8) ou un dict (v7)
        first_file = upload.value[0] if isinstance(upload.value, tuple) else next(iter(upload.value.values()))
        content = first_file.get('content') if isinstance(first_file, dict) else first_file.content
        df_new = pd.read_excel(io.BytesIO(content))
        expected = ['Vb (mL)', 'pH']
        if all(col in df_new.columns for col in expected):
            df = df_new[expected].copy()
        else:
            df = df_new.iloc[:, :2].copy()
            df.columns = expected
        globals()['df'] = df
        update_outputs()
    else:
        with output_table:
            clear_output(wait=True)
            print("Aucun fichier sélectionné.")

# Liaison des callbacks
add_button.on_click(on_add_clicked)
clear_button.on_click(on_clear_clicked)
import_button.on_click(on_import_clicked)
show_deriv.observe(lambda change: update_outputs(), names='value')

# Assemblage de l’interface
ui = widgets.VBox([
    widgets.HBox([vb_input, ph_input, add_button, clear_button]),
    widgets.HBox([upload, import_button]),
    show_deriv,
    widgets.Label("Données :"), output_table,
    widgets.Label("Graphique :"), output_plot
])
display(ui)

# Affichage initial
update_outputs()


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
